# Coding Challenge Practice

## Common Quant Interview Coding Problems

Practice problems with solutions and timing.

In [1]:
import numpy as np
import pandas as pd
from collections import deque
import time

np.random.seed(42)
print("Coding Challenge Practice Ready")

Coding Challenge Practice Ready


## Challenge 1: Moving Average Calculator

**Problem**: Implement a streaming moving average calculator with O(1) update time.

**Time Limit**: 15 minutes

In [2]:
# Solution: Streaming Moving Average
class MovingAverage:
    """O(1) time complexity moving average."""
    
    def __init__(self, window_size):
        self.window_size = window_size
        self.queue = deque(maxlen=window_size)
        self.sum = 0
    
    def next(self, value):
        """Add new value and return current average."""
        if len(self.queue) == self.window_size:
            self.sum -= self.queue[0]  # Remove oldest
        
        self.queue.append(value)
        self.sum += value
        
        return self.sum / len(self.queue)

# Test
ma = MovingAverage(3)
print("Testing MovingAverage(window=3):")
for val in [1, 10, 3, 5, 7]:
    avg = ma.next(val)
    print(f"  Add {val}: MA = {avg:.2f}")

Testing MovingAverage(window=3):
  Add 1: MA = 1.00
  Add 10: MA = 5.50
  Add 3: MA = 4.67
  Add 5: MA = 6.00
  Add 7: MA = 5.00


## Challenge 2: Stock Span Problem

**Problem**: For each day, find how many consecutive days before it had price ≤ current day's price.

**Time Limit**: 20 minutes

In [3]:
# Solution: Stock Span using Stack
def stock_span(prices):
    """Calculate stock span using monotonic stack. O(n) time."""
    n = len(prices)
    spans = [1] * n
    stack = []  # Stack of (index, price)
    
    for i, price in enumerate(prices):
        # Pop all prices smaller than current
        while stack and stack[-1][1] <= price:
            stack.pop()
        
        # Span is distance to previous greater element
        if not stack:
            spans[i] = i + 1
        else:
            spans[i] = i - stack[-1][0]
        
        stack.append((i, price))
    
    return spans

# Test
prices = [100, 80, 60, 70, 60, 75, 85]
spans = stock_span(prices)
print(f"Prices: {prices}")
print(f"Spans:  {spans}")

Prices: [100, 80, 60, 70, 60, 75, 85]
Spans:  [1, 1, 1, 2, 1, 4, 6]


## Challenge 3: Best Time to Buy and Sell Stock

**Problem**: Find maximum profit from at most k transactions.

**Time Limit**: 25 minutes

In [4]:
# Solution: Max Profit with k Transactions (DP)
def max_profit_k_transactions(prices, k):
    """Find max profit with at most k transactions."""
    n = len(prices)
    if n < 2 or k == 0:
        return 0
    
    # If k is large enough, use greedy approach
    if k >= n // 2:
        return sum(max(prices[i+1] - prices[i], 0) for i in range(n-1))
    
    # DP approach
    # dp[t][i] = max profit using at most t transactions up to day i
    dp = [[0] * n for _ in range(k + 1)]
    
    for t in range(1, k + 1):
        max_prev = -prices[0]
        for i in range(1, n):
            dp[t][i] = max(dp[t][i-1], prices[i] + max_prev)
            max_prev = max(max_prev, dp[t-1][i-1] - prices[i])
    
    return dp[k][n-1]

# Test
prices = [3, 2, 6, 5, 0, 3]
for k in [1, 2, 3]:
    profit = max_profit_k_transactions(prices, k)
    print(f"k={k}: Max profit = ${profit}")

k=1: Max profit = $4
k=2: Max profit = $7
k=3: Max profit = $7


## Challenge 4: Implement VWAP

**Problem**: Calculate Volume-Weighted Average Price.

**Time Limit**: 10 minutes

In [5]:
# Solution: VWAP Implementation
def calculate_vwap(prices, volumes):
    """
    Calculate Volume-Weighted Average Price.
    
    VWAP = sum(price * volume) / sum(volume)
    """
    cumulative_pv = np.cumsum(prices * volumes)
    cumulative_v = np.cumsum(volumes)
    vwap = cumulative_pv / cumulative_v
    return vwap

# Test
prices = np.array([100, 101, 99, 102, 100.5])
volumes = np.array([1000, 2000, 500, 1500, 3000])

vwap = calculate_vwap(prices, volumes)
print(f"Prices:  {prices}")
print(f"Volumes: {volumes}")
print(f"VWAP:    {np.round(vwap, 2)}")
print(f"\nFinal VWAP: ${vwap[-1]:.2f}")

Prices:  [100.  101.   99.  102.  100.5]
Volumes: [1000 2000  500 1500 3000]
VWAP:    [100.   100.67 100.43 100.9  100.75]

Final VWAP: $100.75


## Challenge 5: Order Book Matching Engine

**Problem**: Implement a simple order matching engine.

**Time Limit**: 30 minutes

In [6]:
# Solution: Simple Order Book
import heapq

class OrderBook:
    """Simple order book with price-time priority."""
    
    def __init__(self):
        self.bids = []  # Max heap (negate prices)
        self.asks = []  # Min heap
        self.order_id = 0
        self.trades = []
    
    def add_order(self, side, price, quantity):
        """Add a limit order."""
        self.order_id += 1
        order = {'id': self.order_id, 'price': price, 'qty': quantity}
        
        if side == 'buy':
            # Try to match with asks
            while quantity > 0 and self.asks and self.asks[0][0] <= price:
                ask_price, _, ask_order = heapq.heappop(self.asks)
                trade_qty = min(quantity, ask_order['qty'])
                self.trades.append({
                    'price': ask_price,
                    'qty': trade_qty,
                    'buyer': self.order_id,
                    'seller': ask_order['id']
                })
                quantity -= trade_qty
                ask_order['qty'] -= trade_qty
                if ask_order['qty'] > 0:
                    heapq.heappush(self.asks, (ask_price, ask_order['id'], ask_order))
            
            if quantity > 0:
                order['qty'] = quantity
                heapq.heappush(self.bids, (-price, self.order_id, order))
        
        else:  # sell
            while quantity > 0 and self.bids and -self.bids[0][0] >= price:
                neg_bid_price, _, bid_order = heapq.heappop(self.bids)
                bid_price = -neg_bid_price
                trade_qty = min(quantity, bid_order['qty'])
                self.trades.append({
                    'price': bid_price,
                    'qty': trade_qty,
                    'buyer': bid_order['id'],
                    'seller': self.order_id
                })
                quantity -= trade_qty
                bid_order['qty'] -= trade_qty
                if bid_order['qty'] > 0:
                    heapq.heappush(self.bids, (-bid_price, bid_order['id'], bid_order))
            
            if quantity > 0:
                order['qty'] = quantity
                heapq.heappush(self.asks, (price, self.order_id, order))
        
        return self.order_id
    
    def get_best_bid(self):
        return -self.bids[0][0] if self.bids else None
    
    def get_best_ask(self):
        return self.asks[0][0] if self.asks else None

# Test
ob = OrderBook()

print("Adding orders...")
ob.add_order('buy', 100, 10)
ob.add_order('sell', 102, 5)
ob.add_order('buy', 99, 20)
ob.add_order('sell', 100, 15)  # Should match

print(f"\nBest Bid: ${ob.get_best_bid()}")
print(f"Best Ask: ${ob.get_best_ask()}")
print(f"\nTrades executed: {len(ob.trades)}")
for trade in ob.trades:
    print(f"  {trade['qty']} @ ${trade['price']}")

Adding orders...

Best Bid: $99
Best Ask: $100

Trades executed: 1
  10 @ $100


## 🎯 Timing Tips

| Problem Type | Target Time | Key Techniques |
|-------------|-------------|----------------|
| Basic Array | 10-15 min | Two pointers, sliding window |
| Stack/Queue | 15-20 min | Monotonic stack |
| Dynamic Programming | 20-30 min | State definition, transitions |
| Data Structure Design | 25-35 min | Hash maps, heaps |